This notebook contains examples of how to work with [YSON documents](https://ytsaurus.tech/docs/en/user-guide/storage/yson-docs) on YTsaurus.

Notebooks demonstrates how to:
1. Create a document.
2. Write content to a document.
3. Partially update a document.
4. Partially read a document.


Documents can be used to store small amount of data, for example:
1. Common configuration for operation's runner.
2. Debug metadata about ML training run.
3. Settings for YT-based service (like CHYT).

The document behaves as a whole in terms of Cypress-specific features:
* locks
* owners
* revisions
* creation_time, modification_time and expiration_time
* attributes
* and other features

### Limits

* RPS should be < 1
* Single document should be about some KB.
* The total volume of all user documents must not exceed <10MB.



In [1]:
from yt import wrapper as yt
import uuid

## Create a base directory for examples

In [3]:
working_dir = f"//tmp/examples/documents_{uuid.uuid4()}"
yt.create("map_node", working_dir, recursive=True)
print(working_dir)

//tmp/examples/documents_1510bde7-162f-483a-b902-b71011ff6848


## Example

Let's create an empty document and wright some data.

In [6]:
document_path = f"{working_dir}/document"
yt.create("document", document_path)

'307d-dfa3a-134401a5-c1f925d7'

In [7]:
yt.set(document_path, {"data": {"ytsaurus": ["master", "proxies", "scheduler", "exec nodes", "data nodes"]}})
yt.get(document_path)

{'data': {'ytsaurus': ['master', 'proxies', 'scheduler', 'exec nodes', 'data nodes']}}

The document can be partially readed and updated.

In [9]:
yt.get(f"{document_path}/data/ytsaurus")

['master', 'proxies', 'scheduler', 'exec nodes', 'data nodes']

In [10]:
yt.set(f"{document_path}/object_class", "Safe")
yt.get(document_path)

{'data': {'ytsaurus': ['master', 'proxies', 'scheduler', 'exec nodes', 'data nodes']}, 'object_class': 'Safe'}

The document can also be completely rewritten.

In [12]:
yt.set(document_path, "[DATA EXPUNGED]")
yt.get(document_path)

'[DATA EXPUNGED]'